# Install packages and dependencies

In [3]:
!pip install datasets
!pip install evaluate
!pip install accelerate -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.8 MB/s eta 0:00:00


In [ ]:
import torch
from tqdm.auto import tqdm

# Load image processor

In [ ]:
from transformers import ViTImageProcessor, ViTForImageClassification
device = "cuda"

In [ ]:
model_name = "google/vit-base-patch16-224"
image_processor = ViTImageProcessor.from_pretrained(model_name)

# Load and preprocess dataset

In [ ]:
mean = 0
std = 30
n_noised_img = 9

In [ ]:
from noise_addition import GaussianNoise
noise_helper = GaussianNoise(mean, std, n_noised_img)

In [6]:
from datasets import load_dataset

dataset = load_dataset("zh-plus/tiny-imagenet")

training_ds = dataset["train"].shuffle().train_test_split(0.2)
val_ds = dataset["valid"]

features = training_ds["train"].features
labels = features["label"].names
num_labels = len(labels)

print(num_labels)

In [ ]:
training_ds["train"] = noise_helper.add_noise_data(training_ds["train"])
training_ds["test"] = noise_helper.add_noise_data(training_ds["test"])

In [ ]:
def transform(examples):
  # convert all images to RGB format, then preprocessing it using our image processor
  inputs = image_processor([img.convert("RGB") for img in examples["image"]], return_tensors="pt")

  # we also shouldn't forget about the labels
  inputs["labels"] = torch.tensor(examples["label"])

  return inputs

# use the with_transform() method to apply the transform to the dataset on the fly during training
training_ds = training_ds.with_transform(transform)

# Load and train classifier

In [ ]:
model = ViTForImageClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    ignore_mismatched_sizes=True,
)

In [ ]:
def collate_fn(batch):
  return {
      "pixel_values": torch.stack([x["pixel_values"] for x in batch]),
      "labels": torch.tensor([x["labels"] for x in batch]),
  }

In [ ]:
from evaluate import load
import numpy as np

# load the accuracy and f1 metrics from the evaluate module
accuracy = load("accuracy")
f1 = load("f1")

def compute_metrics(eval_pred):
  # compute the accuracy and f1 scores & return them
  accuracy_score = accuracy.compute(predictions=np.argmax(eval_pred.predictions, axis=1), references=eval_pred.label_ids)
  f1_score = f1.compute(predictions=np.argmax(eval_pred.predictions, axis=1), references=eval_pred.label_ids, average="macro")
  return {**accuracy_score, **f1_score}

In [ ]:
# defining the training arguments
training_args = TrainingArguments(
  output_dir=f"trained_classifier", # output directory
  per_device_train_batch_size=32, # batch size per device during training
  eval_strategy="steps",    # evaluation strategy to adopt during training
  num_train_epochs=3,             # total number of training epochs
  # fp16=True,                    # use mixed precision
  save_steps=200,                # number of update steps before saving checkpoint
  eval_steps=200,                # number of update steps before evaluating
  logging_steps=200,             # number of update steps before logging
  save_total_limit=2,             # limit the total amount of checkpoints on disk
  remove_unused_columns=False,    # remove unused columns from the dataset
  push_to_hub=False,              # do not push the model to the hub
  report_to='tensorboard',        # report metrics to tensorboard
  load_best_model_at_end=True,    # load the best model at the end of training
)

In [ ]:
# Create Trainer
from transformers import Trainer

trainer = Trainer(
    model=model,                        # the instantiated 🤗 Transformers model to be trained
    args=training_args,                 # training arguments, defined above
    data_collator=collate_fn,           # the data collator that will be used for batching
    compute_metrics=compute_metrics,    # the metrics function that will be used for evaluation
    train_dataset=training_ds["train"], # the training dataset
    eval_dataset=training_ds["test"],  # the evaluation dataset
    tokenizer=image_processor,          # the processor that will be used for preprocessing the images
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate(valid_ds["test"])

In [ ]:
model = trainer.model

# LoEn Evaluation

In [ ]:
from cal_energy import cal_energy_score

In [ ]:
n_sample = 64

In [ ]:
#feed image to model to get the logits before softmax
def model_forward(x):
    with torch.no_grad():
        output = model(x, output_hidden_states=True)
    logits =  output.logits
    return logits

In [ ]:
sampler = NoiseAdditon(mean, std, n_sample)

In [ ]:
def preprocess(img, label):
    noised_imgs = sampler.add_noise(image)
    imgs = [img] + noised_img
    labels = [label] * (len(imgs))
    preprocessed = transform(
        {
            "images": imgs,
            "label": labels,
        }
    )
    return preprocessed

In [ ]:
y_score = torch.tensor([])
y_true = torch.tensor([])

for data in tqdm(ds):
    pixel_values, label = preprocess(data["image"], data["label"])

    logits = model_forward(pixel_values)
    pred = logits[0].argmax(dim=-1, keepdim=True)

    y_score = torch.concat([y_score, cal_energy_score(logits).cpu()])
    y_true = torch.concat([y_true, label != pred.cpu()])

In [ ]:
import numpy as np
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
fpr, tpr, thresholds = roc_curve(y_true, y_score)
roc_auc = auc(fpr, tpr)
# Plot the ROC curve
plt.figure()
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--', label='No Skill')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.show()